In [206]:
user_business_pro.shape # row : user, column = business

(45770, 10968)

In [ ]:
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation, cosine
import time


In [211]:
def findksimilarusers(user_id, ratings, metric = 'cosine', k=1):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
    model_knn.fit(ratings)

    distances, indices = model_knn.kneighbors(ratings.iloc[user_id-1, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
    print('{0} most similar users for User {1}:\n'.format(k,user_id))
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;

        else:
            print('{0}: User {1}, with similarity of {2}'.format(i, indices.flatten()[i]+1, similarities.flatten()[i]))
            
    return similarities,indices

In [214]:
start = time.time()
similarities,indices = findksimilarusers(1, pd.DataFrame(user_business_pro), metric='cosine', k =5)
end = time.time()
print('time used:', start - end)

5 most similar users for User 1:

1: User 22361, with similarity of 0.9999969791877369
2: User 11374, with similarity of 0.9999964037324052
3: User 43616, with similarity of 0.9999962657738
4: User 15375, with similarity of 0.9999961354682665
5: User 13922, with similarity of 0.9999961131844802
time used: -114.18974685668945


In [217]:
#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, ratings, metric = 'cosine', k=5):
    prediction=0
    similarities, indices=findksimilarusers(user_id, ratings,metric, k) #similar users based on cosine similarity
    mean_rating = ratings.loc[user_id-1,:].mean() #to adjust for zero based indexing
    sum_wt = np.sum(similarities)-1
    product=1
    wtd_sum = 0 
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;
        else: 
            ratings_diff = ratings.iloc[indices.flatten()[i],item_id-1]-np.mean(ratings.iloc[indices.flatten()[i],:])
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product
    
    prediction = int(round(mean_rating + (wtd_sum/sum_wt)))
    print('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [220]:
start = time.time()
predict_userbased(3,4,pd.DataFrame(user_business_pro));
end = time.time()
print('time used:', start - end)

5 most similar users for User 3:

1: User 30313, with similarity of 0.9999542862521746
2: User 12527, with similarity of 0.9999519626136075
3: User 4158, with similarity of 0.9999449538228273
4: User 43316, with similarity of 0.9999257201648606
5: User 23083, with similarity of 0.999923507290708

Predicted rating for user 3 -> item 4: 0
time used: -87.60992002487183
